# Analyse des Musées de France : Top 10 des Départements avec le Plus Grand Nombre de Musées


Le dataset "Liste et Localisation des Musées de France" disponible sur [data.culture.gouv.fr](https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin) fournit des informations précieuses sur les musées de France, notamment leur localisation, leur nom officiel, et d'autres détails administratifs. Ce dataset est issu du ministère de la Culture et est une ressource clé pour comprendre la richesse culturelle de la France.


L'objectif de cette analyse est de répondre à la question : quels sont les départements français qui comptent le plus grand nombre de musées ? Pour cela, nous allons utiliser le dataset pour compter le nombre de musées par département et classer les résultats dans le top 10 des départements avec le plus de musées.


## Méthodologie

La méthodologie utilisée pour cette analyse consiste à charger le dataset dans un dataframe, puis à utiliser une requête SQL avec DuckDB pour compter le nombre de musées par département. Les résultats sont ensuite triés par ordre décroissant et limités aux 10 premiers départements. Les données sont ensuite visualisées sous forme de graphique à barres pour mettre en évidence les départements avec le plus grand nombre de musées.


Les étapes clés de la méthodologie sont :

- Chargement du dataset
- Requête SQL pour compter les musées par département
- Tri et limitation des résultats
- Visualisation des données


La requête SQL utilisée est la suivante :

La visualisation des données est réalisée à l'aide de la bibliothèque Plotly Express, qui permet de créer un graphique à barres interactif et visuellement attrayant.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "departement" AS dept_name,
  COUNT("identifiant_museofile") AS nb_musees
FROM 
  loaded_dataset
GROUP BY 
  "departement"
ORDER BY 
  nb_musees DESC
LIMIT 10 """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 10 lignes


,dept_name,nb_musees
0,Paris,50
1,Bouches-du-Rhône,41
2,Nord,33
3,Alpes-Maritimes,33
4,Seine-Maritime,32


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est une surcouche de Plotly, idéale pour créer des visualisations interactives de données. Elle est adaptée ici pour représenter un graphique à barres triées des départements avec le plus grand nombre de musées, offrant une visualisation claire et interactive. Cela permet une exploration facile des données avec des informations supplémentaires au survol.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px

df_sorted = df.sort_values('nb_musees', ascending=False)

dataviz = px.bar(
    df_sorted,
    x='dept_name',
    y='nb_musees',
    text_auto=False,
    title='Top 10 des départements avec le plus grand nombre de musées',
    labels={'dept_name': 'Département',
            'nb_musees': 'Nombre de musées'},
    color_discrete_sequence=['#1f77b4']
)

dataviz.update_layout(
    xaxis={'categoryorder':'total descending'},
    xaxis_tickangle=-45,
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=100),
    font=dict(family='Arial', size=12),
    hovermode='x unified'
)

dataviz.update_traces(
    hovertemplate='<b>%{x}</b><br>Nombre de musées : %{y}<extra></extra>'
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250818_204609.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250818_204609.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250818_204609.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250818_204609.png


--> Image Plotly sauvegardée avec succès.
